In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_d0dc69483f8a4de1ab6c1df09fbbadbc = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='jq-L-7BvvoaQ2eUnOB2WhhlSTfAITigHiF0vI4CCsWVD',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_d0dc69483f8a4de1ab6c1df09fbbadbc.get_object(Bucket='usecases-donotdelete-pr-x1wobylqzjzfnc',Key='google_review_ratings.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df= pd.read_csv(body)
df.head()



churches  resorts  beaches  parks  museums   zoo  fastfoods  viewpoints  \
0       0.0     3.63     3.65    5.0      5.0  2.33        1.7         0.0   
1       0.0     3.63     3.65    5.0      5.0  2.33        1.7         0.0   
2       0.0     3.63     3.63    5.0      5.0  2.33        1.7         0.0   
3       0.5     3.63     3.63    5.0      5.0  2.33        1.7         0.0   
4       0.0     3.63     3.63    5.0      5.0  2.33        1.7         0.0   

   dependentvariable  
0            2.66375  
1            2.66375  
2            2.66125  
3            2.72375  
4            2.66125

In [5]:
X = df.iloc[:, :-1].values
y = df.iloc[:, 8].values

In [6]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [8]:
y_pred=regressor.predict([[0,3.63,3.65,5,5,2.33,1.7,0]])

In [9]:
y_pred

array([ 2.66472458])

In [10]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

2019-04-24 09:37:43,997 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.


In [11]:
wml_credentials = {
  "username": "5d84c453-2449-482b-9784-07bfc87097ee",
  "password": "9f6d627a-1f89-47a9-9fa3-15405257458d",
  "instance_id": "8ac3af8b-681b-4024-9166-311eb6f50447",
  "url": "https://us-south.ml.cloud.ibm.com"
}

In [12]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "Sasidhar Pokuri", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "sasipv28@gmail.com", 
               client.repository.ModelMetaNames.NAME: "google reviews"
              }

In [14]:
model_artifact =client.repository.store_model(regressor, meta_props=model_props)

In [15]:
published_model_uid = client.repository.get_model_uid(model_artifact)
published_model_uid

'20816301-7736-44f6-ba25-0773f7cac68f'

In [16]:
deployment = client.deployments.create(published_model_uid, name="google reviews")



#######################################################################################

Synchronous deployment creation for uid: '20816301-7736-44f6-ba25-0773f7cac68f' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='7c7c9ea8-7238-4c07-9253-b1a0b711f184'
------------------------------------------------------------------------------------------------




In [17]:
scoring_endpoint = client.deployments.get_scoring_url(deployment)
scoring_endpoint

'https://us-south.ml.cloud.ibm.com/v3/wml_instances/8ac3af8b-681b-4024-9166-311eb6f50447/deployments/7c7c9ea8-7238-4c07-9253-b1a0b711f184/online'